In [1]:
import numpy as np
import pandas as pd
from IPython.display import display
from constants import *

In [2]:
#manually specify dtype for columns for import
sparse_string_dtype = pd.SparseDtype("string")
sparse_int_dtype = pd.SparseDtype("int")
sparse_float_dtype = pd.SparseDtype("float")
sparse_bool_dtype = pd.SparseDtype("bool")

dtypes = {
    'category': 'category',
    'actionType': 'category',
    'service': 'category',
    'reason': 'string',
    'videoID': 'string',
    'userID': 'string',
    'votes': 'int',
    'startTime': 'float',
    'endTime': 'float',
    'locked': 'boolean',
    'incorrectVotes': 'int',
    #'timeSubmitted': 'int',
    'view': 'int',
    'videoDuraction': 'float',
    'hidden': 'int',
    'reputation': 'float',
    'shadowHidden': 'boolean',
    'userAgent': 'string',
    'description': 'string',
    'remove': 'int',
    'published': 'int',
}
# dtypes_sparse = {
#     'reason': sparse_string_dtype,
#     'description': sparse_string_dtype,
#     'hidden': sparse_int_dtype,
#     'votes': sparse_int_dtype,
#     'locked': sparse_int_dtype,
#     'int': sparse_int_dtype,
#     'reputation': sparse_float_dtype,
#     'shadowHidden': sparse_float_dtype,
# }

def convert_timestamp_to_seconds(ts):
    """timeSubmitted is a unix time stamp in ms
    we convert to seconds to avoid integer overflow.
    """
    return int(ts) // 1000

def time_submit_processor(videoPD, sponsorTimePD):
    for 

In [3]:
dfs = {}
for name in mirror_files:
    print(name)
    df = pd.read_csv(data_folder+name, 
                     dtype=dtypes,usecols=lambda col: col not in unwanted_cols,
                     converters={'timeSubmitted':convert_timestamp_to_seconds}
                    )
    # for col in dtypes_sparse:
    #      if col in df.columns:
    #          df[col] = df[col].astype(pd.SparseDtype("string"))
    dfs[name.rstrip('.csv')] = df
    display(df.head())

sponsorTimes.csv


,videoID,startTime,endTime,votes,locked,incorrectVotes,UUID,userID,timeSubmitted,views,category,actionType,videoDuration,hidden,reputation,shadowHidden,userAgent,description
0,uqKGREZs6-w,475.52167,532.208740,302,False,1,622f9270-a2a1-11e9-b210-99c885575bb9,38e7c2af-09f4-4492-bf49-75e443962ccd,1564088876,18060,sponsor,skip,0.0,1,0.0,False,<NA>,<NA>
1,1rb3bMvDdX4,588.94100,631.897770,159,False,1,28aff450-a372-11e9-b256-cb886cabe693,38e7c2af-09f4-4492-bf49-75e443962ccd,1564088876,4430,sponsor,skip,0.0,0,0.0,False,<NA>,<NA>
2,1rb3bMvDdX4,0.00000,6.861597,350,False,1,28ab1250-a372-11e9-b256-cb886cabe693,38e7c2af-09f4-4492-bf49-75e443962ccd,1564088876,11773,sponsor,skip,0.0,0,0.0,False,<NA>,<NA>
3,FfgT6zx4k3Q,446.51013,513.392330,225,False,1,96150fa0-a28a-11e9-b210-99c885575bb9,38e7c2af-09f4-4492-bf49-75e443962ccd,1564088876,3222,sponsor,skip,0.0,1,0.0,False,<NA>,<NA>
4,UjtOGPJ0URM,497.14310,569.153300,-2,False,1,b3237a80-a2c3-11e9-b210-99c885575bb9,38e7c2af-09f4-4492-bf49-75e443962ccd,1564088876,24217,sponsor,skip,0.0,0,0.0,False,<NA>,<NA>


userNames.csv


,userID,userName,locked
0,6da67183d9b987cffe30dd44cfa53a4fb26bf22067c5c9...,RHG,False
1,950f2c3202ff0835ac683a1e97632d71257eeb5a9cc9c7...,Scuddzero,False
2,c4a6408834ac21d6bd8eca3cee787a1b1c3009ffacb1d4...,Geroyuni,False
3,01c70fbb8cebe46f5ca2b81e0c1fe8d490709d86e2624e...,Anonymous SponsorBlock Users,False
4,d45b223dd02b0c6c0019a3d7aa5675245375095fa42345...,Cool People,False


videoInfo.csv


,videoID,channelID,title,published
0,QyTRvbb3gkk,UCr7tNSNf7_aEEh5P-F5mE4A,Kronii Got Friendzoned by Chat but She End Up ...,1634428800
1,qU1Yv58EXcc,UCo_IB5145EVNcf8hw1Kku7w,Game Theory: Minecraft's DARKEST Timeline! (He...,1634428800
2,yKkVHBh9DQk,UCXJkLU1wZVqZjjVe1MuRj-A,TRIPLE RECORD EN GLOBILLOS? 🎈,1633564800
3,7wCZSBOX7eM,UCg83RGdRpwfvoFEuE2zWKZA,Johnny vs. Nickelodeon All-Star Brawl (Sponsored),1633392000
4,VVGjjaWWeRA,UCKBYXp4Xn2I2tL1UL4fpbhw,WOTB | NEW BIG HITTING JAGTIGER PREMIUM!,1634428800


vipUsers.csv


,userID
0,57ddecc5b36813ddb8ea1eba73342c8a783527b884b6eb...
1,c4a6408834ac21d6bd8eca3cee787a1b1c3009ffacb1d4...
2,d261c35ce21b0554c183fc42e2f92bf30609e0540bad8e...
3,963cdc21439055c825860792fa6ef0a48ffc8823f20f31...
4,7b89ea26f77bda8176e655eee86029f28c1e6514b6d6e3...


In [4]:
# investigate missing data
for df in dfs:
    print(f"{df}: length {dfs[df].shape[0]}")
    for col in dfs[df]:
        count = dfs[df][col].isna().sum()
        if count:
            print(f"\t{col}: {count}")

sponsorTime: length 15073175
	videoID: 54
	reputation: 58
	userAgent: 3140631
	description: 14895089
userName: length 381207
	userName: 6640
videoInfo: length 7957893
	videoID: 1
	channelID: 601121
	title: 499447
vipUser: length 114


In [23]:
for df in dfs:
    length = dfs[df].shape[0]
    print(f"{df}: length {length}")
    for col in dfs[df]:
        series = dfs[df][col]
        if pd.api.types.is_numeric_dtype(series) or pd.api.types.is_bool_dtype(series) or series.dtype == sparse_float_dtype or series.dtype == sparse_int_dtype:
            count = (series == 0).sum()
            if count/length >= 0:
                print(f"\t{col}: {count/length}")

sponsorTime: length 15073175
	startTime: 0.19605278914362767
	endTime: 0.006266297578313792
	votes: 0.887230460735711
	locked: 0.9722710046158158
	incorrectVotes: 6.63430232847426e-08
	timeSubmitted: 0.0
	views: 0.26421168731869693
	videoDuration: 0.09204477490641487
	hidden: 0.8705878489435703
	reputation: 0.7474777543550049
	shadowHidden: 0.9460201981334391
userName: length 381207
	locked: 0.9999475350662501
videoInfo: length 7957893
	published: 0.6117296374806749
vipUser: length 114


In [7]:
(dfs['sponsorTime']['videoDuration'] == 0).sum()

1387407

In [11]:
dfs['sponsorTime']['userID'].loc[0]

'38e7c2af-09f4-4492-bf49-75e443962ccd'

In [20]:
length = dfs['sponsorTime'].shape[0]
dfs['sponsorTime'].loc[length//2]

videoID                                                 0xc2uWLUKG0
startTime                                                   164.262
endTime                                                     164.262
votes                                                             0
locked                                                         True
incorrectVotes                                                    1
UUID              2214e25c6ec6832711b3287959c2cf0f941b7ac21089ac...
userID            2008b11634ccabac1ce8bf43a4f028fceeba23931846d3...
timeSubmitted                                            -373315966
views                                                             8
category                                              poi_highlight
actionType                                                      poi
videoDuration                                             12141.003
hidden                                                            0
reputation                                      

In [16]:
(dfs['sponsorTime']['timeSubmitted'] < 0).sum()

6915628

In [21]:
dfs['videoInfo'].loc[dfs['videoInfo']['videoID']=='0xc2uWLUKG0']

,videoID,channelID,title,published
2740960,0xc2uWLUKG0,UCdn5BQ06XqgXoAxIhbqw5Rg,【#ホロ鯖夏祭り】この競馬に 夏の運気を賭けに来た！！！【ホロライブ/白上フブキ】,1661731200


In [ ]:
dfs['sponsorTime'].head()

In [8]:
merged = dfs['sponsorTime'].merge(dfs['videoInfo'], on='videoID', how='left')

In [9]:
merged.head()

,videoID,startTime,endTime,votes,locked,incorrectVotes,UUID,userID,timeSubmitted,views,...,actionType,videoDuration,hidden,reputation,shadowHidden,userAgent,description,channelID,title,published
0,uqKGREZs6-w,475.52167,532.208740,302,False,1,622f9270-a2a1-11e9-b210-99c885575bb9,38e7c2af-09f4-4492-bf49-75e443962ccd,1564088876,18060,...,skip,0.0,1,0.0,False,<NA>,<NA>,UCsXVk37bltHxD1rDPwtNM8Q,Building a Marsbase is a Horrible Idea: Let’s ...,1.549152e+09
1,1rb3bMvDdX4,588.94100,631.897770,159,False,1,28aff450-a372-11e9-b256-cb886cabe693,38e7c2af-09f4-4492-bf49-75e443962ccd,1564088876,4430,...,skip,0.0,0,0.0,False,<NA>,<NA>,NaN,NaN,NaN
2,1rb3bMvDdX4,0.00000,6.861597,350,False,1,28ab1250-a372-11e9-b256-cb886cabe693,38e7c2af-09f4-4492-bf49-75e443962ccd,1564088876,11773,...,skip,0.0,0,0.0,False,<NA>,<NA>,NaN,NaN,NaN
3,FfgT6zx4k3Q,446.51013,513.392330,225,False,1,96150fa0-a28a-11e9-b210-99c885575bb9,38e7c2af-09f4-4492-bf49-75e443962ccd,1564088876,3222,...,skip,0.0,1,0.0,False,<NA>,<NA>,UCsXVk37bltHxD1rDPwtNM8Q,Could Your Phone Hurt You? Electromagnetic Pol...,1.562458e+09
4,UjtOGPJ0URM,497.14310,569.153300,-2,False,1,b3237a80-a2c3-11e9-b210-99c885575bb9,38e7c2af-09f4-4492-bf49-75e443962ccd,1564088876,24217,...,skip,0.0,0,0.0,False,<NA>,<NA>,UCsXVk37bltHxD1rDPwtNM8Q,Why Alien Life Would be our Doom - The Great F...,1.517443e+09


In [10]:
merged.shape

(15073175, 21)

In [11]:
dfs['sponsorTime'].shape

(15073175, 18)

In [12]:
merged['published'].dtype

dtype('float64')

In [15]:
dfs['videoInfo'].loc[dfs['videoInfo']['videoID']=='1rb3bMvDdX4']

,videoID,channelID,title,published


In [16]:
dfs['videoInfo']['published'].dtype

dtype('int32')

In [18]:
merged['published'] = merged['published'].fillna(-1).astype(int)

In [19]:
merged.head()

,videoID,startTime,endTime,votes,locked,incorrectVotes,UUID,userID,timeSubmitted,views,...,actionType,videoDuration,hidden,reputation,shadowHidden,userAgent,description,channelID,title,published
0,uqKGREZs6-w,475.52167,532.208740,302,False,1,622f9270-a2a1-11e9-b210-99c885575bb9,38e7c2af-09f4-4492-bf49-75e443962ccd,1564088876,18060,...,skip,0.0,1,0.0,False,<NA>,<NA>,UCsXVk37bltHxD1rDPwtNM8Q,Building a Marsbase is a Horrible Idea: Let’s ...,1549152000
1,1rb3bMvDdX4,588.94100,631.897770,159,False,1,28aff450-a372-11e9-b256-cb886cabe693,38e7c2af-09f4-4492-bf49-75e443962ccd,1564088876,4430,...,skip,0.0,0,0.0,False,<NA>,<NA>,NaN,NaN,-1
2,1rb3bMvDdX4,0.00000,6.861597,350,False,1,28ab1250-a372-11e9-b256-cb886cabe693,38e7c2af-09f4-4492-bf49-75e443962ccd,1564088876,11773,...,skip,0.0,0,0.0,False,<NA>,<NA>,NaN,NaN,-1
3,FfgT6zx4k3Q,446.51013,513.392330,225,False,1,96150fa0-a28a-11e9-b210-99c885575bb9,38e7c2af-09f4-4492-bf49-75e443962ccd,1564088876,3222,...,skip,0.0,1,0.0,False,<NA>,<NA>,UCsXVk37bltHxD1rDPwtNM8Q,Could Your Phone Hurt You? Electromagnetic Pol...,1562457600
4,UjtOGPJ0URM,497.14310,569.153300,-2,False,1,b3237a80-a2c3-11e9-b210-99c885575bb9,38e7c2af-09f4-4492-bf49-75e443962ccd,1564088876,24217,...,skip,0.0,0,0.0,False,<NA>,<NA>,UCsXVk37bltHxD1rDPwtNM8Q,Why Alien Life Would be our Doom - The Great F...,1517443200


In [22]:
merged['channelID'].isna().sum()

3096982

In [24]:
(merged['timeSubmitted'].loc[0]-merged['published'].loc[0])/3600

4149.1322222222225